In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/0206dacon/sample_submission.csv
/kaggle/input/0206dacon/train.csv
/kaggle/input/0206dacon/test.csv


In [2]:
# ======================================================================================
# [Tier 1] DeepInsight Pipeline for Kaggle
# ======================================================================================
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import copy
import time
import matplotlib.pyplot as plt

# --------------------------------------------------------------------------------------
# [1] 데이터 경로 자동 탐색 & 로드
# --------------------------------------------------------------------------------------
def load_kaggle_data():
    # 캐글의 일반적인 데이터 경로 탐색
    base_path = '/kaggle/input'
    train_path, test_path = None, None
    
    # input 폴더 하위를 뒤져서 train.csv/test.csv 찾기
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == 'train.csv':
                train_path = os.path.join(root, file)
            elif file == 'test.csv':
                test_path = os.path.join(root, file)
    
    if train_path is None or test_path is None:
        # 못 찾았을 경우 (업로드 필요)
        print("❌ 데이터 파일을 찾을 수 없습니다. [Add Data]로 train.csv, test.csv를 추가해주세요.")
        return None, None
    
    print(f"📂 Train Data Found: {train_path}")
    print(f"📂 Test Data Found: {test_path}")
    return pd.read_csv(train_path), pd.read_csv(test_path)

train, test = load_kaggle_data()

# --------------------------------------------------------------------------------------
# [2] Elite Feature Engineering (정예 변수 생성)
# --------------------------------------------------------------------------------------
def create_elite_features(df):
    data = df.copy()
    epsilon = 1e-6 

    # 1. 의료적 구간화
    age_mapping = {'만18-34세': 26, '만35-37세': 36, '만38-39세': 38.5, '만40-42세': 41, '만43-44세': 43.5, '만45-50세': 47.5}
    if '시술 당시 나이' in data.columns:
        data['나이_수치'] = data['시술 당시 나이'].map(age_mapping)
        data['연령_구간'] = pd.cut(data['나이_수치'], bins=[0, 35, 38, 40, 43, 100], labels=[0, 1, 2, 3, 4]).astype(float).fillna(-1)

    if '수집된 신선 난자 수' in data.columns:
        data['난자개수_구간'] = pd.cut(data['수집된 신선 난자 수'], bins=[-1, 5, 10, 15, 100], labels=[0, 1, 2, 3]).astype(float).fillna(-1)

    # 2. 생물학적 효율성
    if '이식된 배아 수' in data.columns and '수집된 신선 난자 수' in data.columns:
        data['난자_최종_활용률'] = data['이식된 배아 수'] / (data['수집된 신선 난자 수'] + epsilon)
    if '총 생성 배아 수' in data.columns and '혼합된 난자 수' in data.columns:
        data['배아_생성_효율'] = data['총 생성 배아 수'] / (data['혼합된 난자 수'] + epsilon)

    # 3. 결측치 정보화
    if '난자 해동 경과일' in data.columns:
        data['신선배아_사용'] = data['난자 해동 경과일'].isnull().astype(int)
        data['난자 해동 경과일'] = data['난자 해동 경과일'].fillna(-1)

    return data

print("⚙️ Generating Elite Features...")
train_fe = create_elite_features(train)
test_fe = create_elite_features(test)

# 데이터 정제 및 스케일링
target = '임신 성공 여부'
X = train_fe.drop(columns=['ID', target])
y = train_fe[target]
X_test = test_fe.drop(columns=['ID'])

# 인코딩 & 스케일링
cat_cols = X.select_dtypes(include=['object']).columns
for col in cat_cols:
    le = LabelEncoder()
    combined = pd.concat([X[col], X_test[col]]).astype(str)
    le.fit(combined)
    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

X = X.fillna(0)
X_test = X_test.fillna(0)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# --------------------------------------------------------------------------------------
# [3] DeepInsight Transformer
# --------------------------------------------------------------------------------------
class DeepInsightTransformer:
    def __init__(self, feature_names, resolution=64, perplexity=5, random_state=42):
        self.feature_names = feature_names
        self.resolution = resolution
        self.perplexity = perplexity
        self.random_state = random_state
        self.coords = None

    def fit(self, X):
        print("🧩 t-SNE running (Mapping features to pixels)...")
        # n_iter 제거 (기본값 사용)
        tsne = TSNE(n_components=2, perplexity=self.perplexity, metric='cosine', random_state=self.random_state)
        embedding = tsne.fit_transform(X.T)
        
        min_coords = embedding.min(axis=0)
        max_coords = embedding.max(axis=0)
        self.coords = np.floor((embedding - min_coords) / (max_coords - min_coords) * (self.resolution - 1)).astype(int)
        return self

    def transform(self, X):
        num_samples = X.shape[0]
        images = np.zeros((num_samples, self.resolution, self.resolution))
        for i, (r, c) in enumerate(self.coords):
            images[:, r, c] = X[:, i]
        return images

# 변환 수행
transformer = DeepInsightTransformer(feature_names=X.columns, resolution=64, perplexity=5)
transformer.fit(X_scaled)
print("🖼️ Transforming to Images...")
X_img = transformer.transform(X_scaled)
X_test_img = transformer.transform(X_test_scaled)

# --------------------------------------------------------------------------------------
# [4] CNN Training (ResNet18)
# --------------------------------------------------------------------------------------
class InfertilityImageDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        img = np.stack([img, img, img], axis=-1)
        img = (img * 255).astype(np.uint8)
        if self.transform: img = self.transform(img)
        if self.labels is not None: return img, torch.tensor(self.labels[idx], dtype=torch.float32)
        return img

def get_model():
    model = models.resnet18(pretrained=True)
    model.fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(model.fc.in_features, 1))
    return model

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Training on {device} (GPU Check!)")

SEED = 42
N_FOLDS = 5
BATCH_SIZE = 128
EPOCHS = 10 

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

img_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 5-Fold Training Loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X_img, y)):
    print(f"\n===== Fold {fold+1}/{N_FOLDS} =====")
    
    train_ds = InfertilityImageDataset(X_img[train_idx], y.iloc[train_idx].values, transform=img_transforms)
    val_ds = InfertilityImageDataset(X_img[val_idx], y.iloc[val_idx].values, transform=img_transforms)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    model = get_model().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)
    
    best_auc = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(EPOCHS):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
        model.eval()
        val_targets, val_probs = [], []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                val_probs.append(torch.sigmoid(outputs).cpu().numpy())
                val_targets.append(targets.cpu().numpy())
                
        val_probs = np.concatenate(val_probs).ravel()
        val_targets = np.concatenate(val_targets).ravel()
        val_auc = roc_auc_score(val_targets, val_probs)
        
        print(f"  Epoch {epoch+1} | Val AUC: {val_auc:.5f}")
        scheduler.step(val_auc)
        
        if val_auc > best_auc:
            best_auc = val_auc
            best_model_wts = copy.deepcopy(model.state_dict())
            
    # Inference (Best Model)
    model.load_state_dict(best_model_wts)
    model.eval()
    
    # OOF
    with torch.no_grad():
        oof_fold = []
        for inputs, _ in val_loader:
            inputs = inputs.to(device)
            oof_fold.append(torch.sigmoid(model(inputs)).cpu().numpy())
    oof_preds[val_idx] = np.concatenate(oof_fold).ravel()
    
    # Test Prediction
    test_ds = InfertilityImageDataset(X_test_img, transform=img_transforms)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    fold_test = []
    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            fold_test.append(torch.sigmoid(model(inputs)).cpu().numpy())
    test_preds += np.concatenate(fold_test).ravel() / N_FOLDS
    
    print(f"✨ Fold {fold+1} Best AUC: {best_auc:.5f}")

print(f"\n🏆 Overall DeepInsight AUC: {roc_auc_score(y, oof_preds):.5f}")

# 결과 저장 (Output 폴더에 저장됨)
submission = pd.read_csv(test_path) if 'test_path' in locals() else pd.DataFrame({'ID': range(len(test_preds))}) # Fallback
# 실제 제출 파일 로드 시도
try:
    sub_df = pd.read_csv('/kaggle/input/sample_submission.csv') # 대회마다 경로 다를 수 있음
except:
    # 경로 못 찾으면 test 데이터 ID 사용
    sub_df = pd.DataFrame({'ID': test['ID']})

sub_df['probability'] = test_preds
sub_df.to_csv('submission_deepinsight_kaggle.csv', index=False)
print("💾 Saved: submission_deepinsight_kaggle.csv")

📂 Train Data Found: /kaggle/input/0206dacon/train.csv
📂 Test Data Found: /kaggle/input/0206dacon/test.csv
⚙️ Generating Elite Features...
🧩 t-SNE running (Mapping features to pixels)...
🖼️ Transforming to Images...
🚀 Training on cuda (GPU Check!)

===== Fold 1/5 =====


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 220MB/s]


  Epoch 1 | Val AUC: 0.73316
  Epoch 2 | Val AUC: 0.73408
  Epoch 3 | Val AUC: 0.73289
  Epoch 4 | Val AUC: 0.73606
  Epoch 5 | Val AUC: 0.73486
  Epoch 6 | Val AUC: 0.73694
  Epoch 7 | Val AUC: 0.73669
  Epoch 8 | Val AUC: 0.73700
  Epoch 9 | Val AUC: 0.73613
  Epoch 10 | Val AUC: 0.73610
✨ Fold 1 Best AUC: 0.73700

===== Fold 2/5 =====


...

  Epoch 1 | Val AUC: 0.73526
  Epoch 2 | Val AUC: 0.73645
  Epoch 3 | Val AUC: 0.73735
  Epoch 4 | Val AUC: 0.73857
  Epoch 5 | Val AUC: 0.73779
  Epoch 6 | Val AUC: 0.73744
  Epoch 7 | Val AUC: 0.73808
  Epoch 8 | Val AUC: 0.73964
  Epoch 9 | Val AUC: 0.74032
  Epoch 10 | Val AUC: 0.73856
✨ Fold 3 Best AUC: 0.74032

===== Fold 4/5 =====


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  Epoch 1 | Val AUC: 0.73089
  Epoch 2 | Val AUC: 0.73352
  Epoch 3 | Val AUC: 0.73447
  Epoch 4 | Val AUC: 0.73576
  Epoch 5 | Val AUC: 0.73586
  Epoch 6 | Val AUC: 0.73548
  Epoch 7 | Val AUC: 0.73562
  Epoch 8 | Val AUC: 0.73572
  Epoch 9 | Val AUC: 0.73655
  Epoch 10 | Val AUC: 0.73582
✨ Fold 4 Best AUC: 0.73655

===== Fold 5/5 =====


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  Epoch 1 | Val AUC: 0.73483
  Epoch 2 | Val AUC: 0.73519
  Epoch 3 | Val AUC: 0.73716
  Epoch 4 | Val AUC: 0.73808
  Epoch 5 | Val AUC: 0.73832
  Epoch 6 | Val AUC: 0.73369
  Epoch 7 | Val AUC: 0.73894
  Epoch 8 | Val AUC: 0.73963
  Epoch 9 | Val AUC: 0.73986
  Epoch 10 | Val AUC: 0.73806
✨ Fold 5 Best AUC: 0.73986

🏆 Overall DeepInsight AUC: 0.73849
💾 Saved: submission_deepinsight_kaggle.csv


In [3]:
from IPython.display import FileLink
FileLink('submission_deepinsight_kaggle.csv')

/kaggle/working/submission_deepinsight_kaggle.csv

In [4]:
import os
print("📂 현재 폴더 파일 목록:")
print(os.listdir("."))  # 현재 위치의 모든 파일 출력

📂 현재 폴더 파일 목록:
['submission_deepinsight_kaggle.csv', '__notebook__.ipynb']


In [5]:
import pandas as pd
import os

# 1. 메모리에 예측값(test_preds)이 살아있는지 확인
try:
    print(f"🚑 구조 작업 시작... 데이터 길이: {len(test_preds)}")
    
    # 살아있다면 즉시 저장 (절대 경로 사용)
    # 캐글의 제출 양식 파일 로드 시도
    try:
        sub = pd.read_csv('../input/sample_submission.csv')
    except:
        # 못 찾으면 경로 검색
        for root, dirs, files in os.walk('/kaggle/input'):
            for file in files:
                if 'sample_submission.csv' in file:
                    sub = pd.read_csv(os.path.join(root, file))
                    break
    
    # 데이터 덮어쓰기 및 저장
    sub['probability'] = test_preds
    sub.to_csv('/kaggle/working/rescue_submission.csv', index=False)
    
    print("✅ [성공] 메모리에서 데이터를 건져냈습니다!")
    print("👉 '/kaggle/working/rescue_submission.csv' 파일을 찾아보세요.")
    
    # 다운로드 링크 생성
    from IPython.display import FileLink
    display(FileLink(r'rescue_submission.csv'))

except NameError:
    print("❌ [실패] 변수가 메모리에서 지워졌습니다. (커널이 재시작된 것 같습니다)")
except Exception as e:
    print(f"⚠️ [오류] {e}")

🚑 구조 작업 시작... 데이터 길이: 90067
✅ [성공] 메모리에서 데이터를 건져냈습니다!
👉 '/kaggle/working/rescue_submission.csv' 파일을 찾아보세요.


/kaggle/working/rescue_submission.csv